In [23]:
from modified_env import CustomEnv
import gymnasium as gym
from stable_baselines3 import PPO, A2C
import pygame

In [24]:
screen = pygame.display.set_mode((800, 600))
env = CustomEnv(screen)

In [3]:
ppo_mlp_model = PPO("MlpPolicy", env, verbose=1, batch_size=4)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/home/koala/.local/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Access the neural network from the policy of the PPO model
neural_network = ppo_mlp_model.policy.to('cuda')

# Get the parameters of the neural network
parameters = list(neural_network.parameters())

# Print out the layers and their corresponding parameters"
for i, param in enumerate(parameters):
    print(f"Layer {i}: {param.shape}")

# Optionally, you can print the entire neural network architecture
print(neural_network)

Layer 0: torch.Size([64, 1440000])
Layer 1: torch.Size([64])
Layer 2: torch.Size([64, 64])
Layer 3: torch.Size([64])
Layer 4: torch.Size([64, 1440000])
Layer 5: torch.Size([64])
Layer 6: torch.Size([64, 64])
Layer 7: torch.Size([64])
Layer 8: torch.Size([5, 64])
Layer 9: torch.Size([5])
Layer 10: torch.Size([1, 64])
Layer 11: torch.Size([1])
ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=1440000, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=1440000, out_features=64, bias=True)
      (1): Tanh()
 

In [6]:
a2c_mlp_model = A2C("MlpPolicy", env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [22]:
# Access the neural network from the policy of the PPO model
neural_network = a2c_mlp_model.policy.to('cuda')

# Get the parameters of the neural network
parameters = list(neural_network.parameters())

# Print out the layers and their corresponding parameters
for i, param in enumerate(parameters):
    print(f"Layer {i}: {param.shape}")

# Optionally, you can print the entire neural network architecture
print(neural_network)

NameError: name 'a2c_mlp_model' is not defined

In [9]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch 
import torch.nn as nn

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
# Define the custom CNN feature extractor
class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
        super().__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Flatten(),
        )

        with torch.no_grad():
            n_flatten = self.cnn(torch.as_tensor(observation_space.sample()[None]).float()).shape[1]
        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))

In [17]:
# Set up PPO with the custom CNN
policy_kwargs = {
    "features_extractor_class": CustomCNN,
    "features_extractor_kwargs": {"features_dim": 128}
}

In [16]:
a2c_cnn_model = A2C("CnnPolicy", env, verbose=1, policy_kwargs=policy_kwargs)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [15]:
# Access the neural network from the policy of the PPO model
neural_network = a2c_cnn_model.policy.to('cuda')

# Get the parameters of the neural network
parameters = list(neural_network.parameters())

# Print out the layers and their corresponding parameters
for i, param in enumerate(parameters):
    print(f"Layer {i}: {param.shape}")

# Optionally, you can print the entire neural network architecture
print(neural_network)

Layer 0: torch.Size([32, 3, 8, 8])
Layer 1: torch.Size([32])
Layer 2: torch.Size([64, 32, 4, 4])
Layer 3: torch.Size([64])
Layer 4: torch.Size([128, 457856])
Layer 5: torch.Size([128])
Layer 6: torch.Size([64, 128])
Layer 7: torch.Size([64])
Layer 8: torch.Size([64, 64])
Layer 9: torch.Size([64])
Layer 10: torch.Size([64, 128])
Layer 11: torch.Size([64])
Layer 12: torch.Size([64, 64])
Layer 13: torch.Size([64])
Layer 14: torch.Size([5, 64])
Layer 15: torch.Size([5])
Layer 16: torch.Size([1, 64])
Layer 17: torch.Size([1])
ActorCriticCnnPolicy(
  (features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=457856, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): CustomCNN(
    (cnn): Sequential(
    

In [21]:
import torchviz
from torch import save
import matplotlib.pyplot as plt

# Access the neural network from the policy of the PPO model
neural_network = a2c_mlp_model.policy.to('cuda')# Move to CPU if necessary

# Dummy input for the visualization
dummy_input = torch.randn(1, *env.observation_space.shape)

# Generate the computational graph
graph = torchviz.make_dot(neural_network(dummy_input), params=dict(neural_network.named_parameters()))

# Save the graph as a JPEG file
graph.render("neural_network_architecture", format="jpg")

# Display the graph
graph.view()


NameError: name 'a2c_mlp_model' is not defined

In [18]:
ppo_cnn_model = A2C("CnnPolicy", env, verbose=1, policy_kwargs=policy_kwargs)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [19]:
# Access the neural network from the policy of the PPO model
neural_network = ppo_cnn_model.policy.to('cuda')

# Get the parameters of the neural network
parameters = list(neural_network.parameters())

# Print out the layers and their corresponding parameters
for i, param in enumerate(parameters):
    print(f"Layer {i}: {param.shape}")

# Optionally, you can print the entire neural network architecture
print(neural_network)

Layer 0: torch.Size([32, 3, 8, 8])
Layer 1: torch.Size([32])
Layer 2: torch.Size([64, 32, 4, 4])
Layer 3: torch.Size([64])
Layer 4: torch.Size([128, 457856])
Layer 5: torch.Size([128])
Layer 6: torch.Size([64, 128])
Layer 7: torch.Size([64])
Layer 8: torch.Size([64, 64])
Layer 9: torch.Size([64])
Layer 10: torch.Size([64, 128])
Layer 11: torch.Size([64])
Layer 12: torch.Size([64, 64])
Layer 13: torch.Size([64])
Layer 14: torch.Size([5, 64])
Layer 15: torch.Size([5])
Layer 16: torch.Size([1, 64])
Layer 17: torch.Size([1])
ActorCriticCnnPolicy(
  (features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=457856, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): CustomCNN(
    (cnn): Sequential(
    